In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.path import Path
import xarray as xr
import math

sys.path.append('..//')
from utils import translate_grid_to_origin, get_grid_angle, rotate_grid

In [ ]:
suffix_particle_files = 'diff_2Cs'
particle_folders =[rf"~//parcels_toolbox//02_output//artful_01_{suffix_particle_files}.zarr",
                  rf"~//parcels_toolbox//02_output//artful_02_{suffix_particle_files}.zarr",
                  rf"~//parcels_toolbox//02_output//artful_03_{suffix_particle_files}.zarr"]

plot_dates = [np.datetime64('2023-09-12 10:00:00'),
              np.datetime64('2023-11-08 11:00:00'),
              np.datetime64('2024-05-01 10:00:00')]

measure_file_paths = [ r"..//coord_sampling//measures_20230912.txt",
                       r"..//coord_sampling//measures_20231108.txt",
                       r"..//coord_sampling//measures_20240501.txt"]

In [ ]:
rotate = True

i_scenario = 2
period_seeding = np.timedelta64(3, 'D')


particle_folder = particle_folders[i_scenario]
plot_date = plot_dates[i_scenario]

date_start_seeding = plot_date - period_seeding

measure_file_path = measure_file_paths[i_scenario]

In [ ]:
str(plot_date)

In [ ]:
from zoneinfo import ZoneInfo 
dt_naive = plot_date.astype('datetime64').astype(datetime)
dt_utc = dt_naive.replace(tzinfo=ZoneInfo("UTC"))
dt_bern = dt_utc.astimezone(ZoneInfo("Europe/Zurich"))


In [ ]:
str_Bern_plot_date = dt_bern.strftime('%Y-%m-%d %H:%M:%S %Z')
str_Bern_plot_date_file_names = dt_bern.strftime('%Y-%m-%d_%H-%M-%S_%Z')
print(str_Bern_plot_date)

In [ ]:
plot_title = f"{period_seeding} old plume"
plot_output_particles_path = rf'.//figures//particles//{str_Bern_plot_date_file_names}_{suffix_particle_files}_{str(period_seeding).replace(" ", "")}.png'
plot_output_path = rf'.//figures//concentration//{str_Bern_plot_date_file_names}_{suffix_particle_files}_{str(period_seeding).replace(" ", "")}.png'
plot_output_path_zoom = rf'.//figures//concentration//{str_Bern_plot_date_file_names}_{suffix_particle_files}_{str(period_seeding).replace(" ", "")}_zoom.png'
csv_output_path = rf'.//csv//concentration//{str_Bern_plot_date_file_names}_{suffix_particle_files}_{str(period_seeding).replace(" ", "")}.csv'

os.makedirs(".//figures//particles", exist_ok=True)
os.makedirs(".//figures//concentration", exist_ok=True)
os.makedirs(".//csv//concentration", exist_ok=True)

# Get particles

In [ ]:
xr_particules = xr.open_zarr(particle_folder) # obs= time index (0=the moment the particule is seeded), trajectory= particle

In [ ]:
arr_time = xr_particules.time.values.flatten()
df_time = pd.Series(arr_time)
df_time = df_time.dropna().drop_duplicates()

In [ ]:
df_time

In [ ]:
xr_particules['age'] = (xr_particules['time'] - xr_particules['time'].isel(obs=0)).dt.total_seconds() / 86400

### Get wwtp and shoreline coordinates

In [ ]:
x_sg_wwtp, y_sg_wwtp = 534650,151350 
x_wwtp, y_wwtp = (885.255248*50, 427.061058*50)
xy_land = np.loadtxt(r"..//..//plot_results//data//shorelines//geneva.txt")

### Filter by initial date

In [ ]:
t_ini = xr_particules.isel(obs=0)['time'].values
pid = xr_particules['trajectory'].values

mask = (t_ini >= np.datetime64(date_start_seeding)) & (t_ini <= np.datetime64(plot_date))
filtered_pid = pid[mask]
filtered_particles = xr_particules.sel(trajectory=filtered_pid)

In [ ]:
filtered_pid

### Select plot date 

In [ ]:
snapshot = filtered_particles.where(filtered_particles['time']==plot_date)
mask_snapshot = ~np.isnan(snapshot.lon.values)

In [ ]:
x_part, y_part, z_part, age_part = snapshot["lon"].values[mask_snapshot], snapshot["lat"].values[mask_snapshot], snapshot["z"].values[mask_snapshot], snapshot["age"].values[mask_snapshot]

### Filter by depth ?

In [ ]:
mask_depth = (z_part < -24) & (z_part > -26)

x_part, y_part, z_part, age_part = x_part[mask_depth], y_part[mask_depth], z_part[mask_depth], age_part[mask_depth]

In [ ]:
print(len(x_part))
print(len(x_part[mask_depth]) - len(x_part))

## Convert particles coordinates to CH1903

In [ ]:
# point of origin in swiss grid 
X0_SG = 500000
Y0_SG = 116500

# second point for tilted grid
X1_SG = 563000
Y1_SG = 138700

angle = get_grid_angle(X0_SG, Y0_SG, X1_SG, Y1_SG)

In [ ]:
# XY on swiss grid 
x_sg = x_part + X0_SG
y_sg = y_part + Y0_SG

In [ ]:
if rotate:
    x_part, y_part = rotate_grid(x_sg, y_sg, X0_SG, Y0_SG, angle)

# Remove particles outside the lake

In [ ]:
polygon = Path(xy_land)
points = np.vstack((x_part, y_part)).T

is_inside = polygon.contains_points(points)

x_inside = x_part[is_inside]
y_inside = y_part[is_inside]
z_inside = z_part[is_inside]
age_inside = age_part[is_inside]

if len(is_inside) != len(x_part):
    print(f'Uhoh, {len(x_part)-len(is_inside)} particules ended up outside the lake...')

## Add measure points

In [ ]:
meas_coordinates = pd.read_csv(measure_file_path, delimiter=';', dtype=None, names=["label", "x", "y", "depths"], encoding='utf-8')
colors = {"2 m": "#ff00e0", "2,25,30.5 m": "red", "25 m":"#fc9803"}

# Aggregate particles on the grid

### Get grid

r_seeding = 50 #[m]
A_seeding = math.pi * r_seeding**2
step = math.sqrt(A_seeding)
print(step)

In [ ]:
step = 50
conc_init = 2000 # particules / cellule

x_grid = np.arange(np.nanmin(x_inside), np.nanmax(x_inside), step)
y_grid = np.arange(np.nanmin(y_inside), np.nanmax(y_inside), step)

### Aggregate particles on grid

In [ ]:
particle_concentration, x_edges, y_edges = np.histogram2d(x_inside, y_inside, bins=[x_grid,y_grid])
print(particle_concentration.max())
print(particle_concentration.min())
particle_concentration[particle_concentration == 0] = np.nan
norm_particle_concentration = particle_concentration / conc_init

# Plot

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
fig = plt.figure(figsize=(6,5.5))
#lake contour
plt.plot(xy_land[:,0], xy_land[:,1], color='black', zorder=1)

#particles
scatter = plt.scatter(x_inside, y_inside, 
                      c=age_inside, cmap='jet', s=2, vmin=0, vmax=8)

cbar = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max')
cbar.set_label(label='$\mathregular{Particle \ age\ [days]}$')

#plt.scatter(x_sg_wwtp, y_sg_wwtp, color='r', zorder=4)
for label in meas_coordinates["depths"].unique():
    subset = meas_coordinates[meas_coordinates["depths"] == label]
    plt.scatter(subset["x"], subset["y"], label=label, marker='x', zorder=3, color=colors[label], linewidths=2)


plt.legend()
plt.title(plot_title)

#plt.xlim(526000,541000)
#plt.ylim(140000,155000)
plt.xlim(524000,544000)
plt.ylim(135000,155000)



plt.xlabel("Lat (km CH1903)")
plt.ylabel("Lon (km CH1903)")

plt.text(0.02, 0.98, f'{str_Bern_plot_date}', transform=plt.gca().transAxes, ha='left', va='top')
plt.grid(False)
plt.tight_layout()
fig.savefig(plot_output_particles_path)

In [ ]:
fig = plt.figure(figsize=(6,5.5))
#lake contour
plt.plot(xy_land[:,0], xy_land[:,1], color='black', zorder=1)

#particles
scatter = plt.scatter(x_inside, y_inside,
                      c=z_inside, cmap='jet', s=2)

cbar = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max')
cbar.set_label(label='$\mathregular{Particle \ depth\ [days]}$')

#plt.scatter(x_sg_wwtp, y_sg_wwtp, color='r', zorder=4)
for label in meas_coordinates["depths"].unique():
    subset = meas_coordinates[meas_coordinates["depths"] == label]
    plt.scatter(subset["x"], subset["y"], label=label, marker='x', zorder=3, color=colors[label], linewidths=2)


plt.legend()
plt.title(plot_title)

#plt.xlim(526000,541000)
#plt.ylim(140000,155000)
plt.xlim(524000,544000)
plt.ylim(135000,155000)



plt.xlabel("Lat (km CH1903)")
plt.ylabel("Lon (km CH1903)")

plt.text(0.02, 0.98, f'{str_Bern_plot_date}', transform=plt.gca().transAxes, ha='left', va='top')
plt.grid(False)
plt.tight_layout()
fig.savefig(plot_output_particles_path)

In [ ]:
fig = plt.figure(figsize=(6,5.5))
# Lake contour
plt.plot(xy_land[:,0], xy_land[:,1], color='black', zorder=1)
# Particles concentration
plt.pcolormesh(x_edges, y_edges, norm_particle_concentration.T, cmap='jet', zorder=2, norm=LogNorm(vmin=1e-4, vmax=1))

cbar = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max')
cbar.set_label(label='$\mathregular{Particle \ concentration\ [-]}$')

#plt.scatter(x_sg_wwtp, y_sg_wwtp, color='r', zorder=4)
# Sampling points
for label in meas_coordinates["depths"].unique():
    subset = meas_coordinates[meas_coordinates["depths"] == label]
    plt.scatter(subset["x"], subset["y"], label=label, marker='x', zorder=3, color=colors[label], linewidths=2)

plt.legend()
plt.title(plot_title)

#plt.xlim(526000,541000)
#plt.ylim(140000,155000)
plt.xlim(524000,544000)
plt.ylim(135000,155000)

plt.xlabel("Lat (km CH1903)")
plt.ylabel("Lon (km CH1903)")

plt.text(0.02, 0.98, f'{str_Bern_plot_date}', transform=plt.gca().transAxes, ha='left', va='top')
plt.grid(False)
plt.tight_layout()
fig.savefig(plot_output_path)

In [ ]:
fig = plt.figure(figsize=(6,5.5))
plt.plot(xy_land[:,0], xy_land[:,1], color='black', zorder=1)
plt.pcolormesh(x_edges,y_edges, norm_particle_concentration.T, cmap='jet', zorder=2, norm=LogNorm(vmin=1e-4, vmax=1))

cbar = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max')
cbar.set_label(label='$\mathregular{Particle \ concentration\ [-]}$')

#plt.scatter(x_sg_wwtp, y_sg_wwtp, color='r', zorder=4)
for label in meas_coordinates["depths"].unique():
    subset = meas_coordinates[meas_coordinates["depths"] == label]
    plt.scatter(subset["x"], subset["y"], label=label, marker='x', zorder=3, color=colors[label], linewidths=2)


plt.legend()
plt.title(plot_title)

plt.xlim(526000,541000)
plt.ylim(140000,155000)

plt.xlabel("Lat (km CH1903)")
plt.ylabel("Lon (km CH1903)")

plt.text(0.02, 0.98, f'{str_Bern_plot_date}', transform=plt.gca().transAxes, ha='left', va='top')
plt.grid(False)
plt.tight_layout()
fig.savefig(plot_output_path_zoom)

In [ ]:
plot_output_path_zoom

# Get particles statistics

In [ ]:
def select_concentrations_from_distance(x_conc, y_conc, concentrations, point_location, max_distance_from_point):
    # Calculate distances using vectorized operations
    distances = np.sqrt((x_conc - point_location['x'])**2 + (y_conc - point_location['y'])**2)
    
    # Select indices where distance is less than max_distance_from_location
    sel_concentrations = np.nan_to_num(concentrations[distances < max_distance_from_point], nan=0.0)
    
    if not sel_concentrations.any():
        sel_concentrations = [0]
    
    # Convert to list if needed
    return list(sel_concentrations)

In [ ]:
search_radius = 300 # meters

In [ ]:
x_edges_grid, y_edges_grid = np.meshgrid(x_edges[:-1], y_edges[:-1])

sel_conc_per_point = []
for index, row in meas_coordinates.iterrows():
    selected_concentrations = select_concentrations_from_distance(x_edges_grid.flatten(), y_edges_grid.flatten(), norm_particle_concentration.T.flatten(), row, search_radius)
    sel_conc_per_point.append(selected_concentrations)

In [ ]:
meas_coordinates['min_conc'] = [min(arr) for arr in sel_conc_per_point]
meas_coordinates['max_conc'] = [max(arr) for arr in sel_conc_per_point]
meas_coordinates['mean_conc'] = [np.mean(arr) for arr in sel_conc_per_point]
meas_coordinates.to_csv(csv_output_path)

In [ ]:
csv_output_path